# NonlinearModel
* Uses LightGBM to estimate a given (user, item) prediction

In [ ]:
task = ""

In [ ]:
import NBInclude: @nbinclude
@nbinclude("TreeModelBase.ipynb")

In [ ]:
train_model(
    [
        explicit_raw_alphas(task)
        implicit_raw_alphas(task)
        nondirectional_raw_alphas
        ["$task/LinearExplicit", "$task/LinearImplicit"]
    ],
    ["$task/LinearExplicit"],
    task,
    ["explicit"],
    false,
    "validation",
    false,
    "$task/NonlinearExplicit";
    λ = nothing,
);

In [ ]:
train_model(
    [
        explicit_raw_alphas(task)
        implicit_raw_alphas(task)
        nondirectional_raw_alphas
        ["$task/LinearExplicit", "$task/LinearImplicit"]
    ],
    String[],
    task,
    ["implicit", "negative"],
    true,
    "validation",
    false,
    "$task/NonlinearImplicit";
    λ = nothing,
);

In [ ]:
# train_model(
#     [
#         explicit_raw_alphas
#         implicit_raw_alphas
#         ptw_raw_alphas
#         nondirectional_raw_alphas
#         ["$task/LinearExplicit", "$task/LinearImplicit", "$task/LinearPtw"]
#     ],
#     String[],
#     ["ptw", "negative"],
#     true,
#     "validation",
#     false,
#     "NonlinearPtw";
#     λ = nothing,
# );

In [ ]:
function save_final_explicit_model()
    function model(split::String, task::String, content::String; raw_splits = true)
        split_fn = raw_splits ? get_raw_split : get_split
        alpha_fn = raw_splits ? read_raw_alpha : read_alpha
        if split == "training"
            return zeros(Float32, length(split_fn(split, task, content).user))
        end
        linear_split = alpha_fn("$task/LinearExplicit", split, task, content).rating
        nonlinear_split = alpha_fn("$task/NonlinearExplicit", split, task, content).rating
        linear_split + nonlinear_split
    end
    write_params(
        Dict("alphas" => ["$task/LinearExplicit", "$task/NonlinearExplicit"]),
        "$task/Explicit",
    )
    write_alpha(
        model,
        "$task/Explicit";
        task = task,
        by_split = true,
        log = true,
        log_alphas = String[],
        log_task = task,
        log_content = "explicit",
        log_splits = ["validation", "test"],
    )
end;

In [ ]:
save_final_explicit_model()